# Проект по SQL

***Описание проекта:***
- *Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.*

- *Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.*
---

### *Подключение к базе данных*

In [40]:
import pandas as pd

from sqlalchemy import create_engine

In [41]:
db_config = {'user': 'praktikum_student',
             'pwd': 'Sdf4$2;d-d30pp',
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,
             'db': 'data-analyst-final-project-db'}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [42]:
# выводим первые строки таблиц

query1 = 'SELECT * FROM books LIMIT 5'
books = pd.io.sql.read_sql(query1, con = engine)

query2 = 'SELECT * FROM authors LIMIT 5'
authors = pd.io.sql.read_sql(query2, con = engine) 

query3 = 'SELECT * FROM publishers LIMIT 5'
publishers = pd.io.sql.read_sql(query3, con = engine)

query4 = 'SELECT * FROM ratings LIMIT 5'
ratings = pd.io.sql.read_sql(query4, con = engine) 

query5 = 'SELECT * FROM reviews LIMIT 5'
reviews = pd.io.sql.read_sql(query5, con = engine) 

In [43]:
display(books, authors, publishers, ratings, reviews)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


---
### *Задачи*

- *Посчитайте, сколько книг вышло после 1 января 2000 года.*

In [44]:
query = '''
            SELECT COUNT(book_id) as amount 
            FROM books 
            WHERE publication_date > '2000-01-01' 
        '''
task1 = pd.io.sql.read_sql(query, con = engine)

print(f'{task1.amount[0]} книг вышло после 1 января 2000 года.')

819 книг вышло после 1 января 2000 года.


- *Для каждой книги посчитайте количество обзоров и среднюю оценку.*

In [45]:
query = '''
    WITH a AS(
                SELECT books.book_id,
                       books.title, 
                       COUNT(reviews.review_id) AS count_reviews
                FROM books LEFT JOIN reviews ON books.book_id = reviews.book_id
                GROUP BY(books.book_id, books.title)), 

         b AS(
                SELECT books.book_id,
                       books.title, 
                       AVG(ratings.rating) AS avg_ratings
                FROM books LEFT JOIN ratings ON books.book_id = ratings.book_id
                GROUP BY(books.book_id, books.title))

    SELECT a.title, 
           a.count_reviews,
           b.avg_ratings
    FROM a INNER JOIN b on a.book_id = b.book_id 
    ORDER BY a.count_reviews, avg_ratings
'''

task2 = pd.io.sql.read_sql(query, con = engine)

display(task2)

,title,count_reviews,avg_ratings
0,The Natural Way to Draw,0,3.000000
1,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
2,Leonardo's Notebooks,0,4.000000
3,Essential Tales and Poems,0,4.000000
4,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
...,...,...,...
995,The Glass Castle,6,4.206897
996,The Book Thief,6,4.264151
997,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
998,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634


- *Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.*

In [46]:
query = ''' 
    WITH a AS(
                SELECT publishers.publisher_id,
                       publisher, 
                       COUNT(book_id) AS count_books
                FROM books INNER JOIN publishers ON books.publisher_id = publishers.publisher_id 
                WHERE num_pages > 50
                GROUP BY publisher, publishers.publisher_id),
                
        b AS(
                SELECT max(count_books) as max_count_books
                FROM a)
  
    SELECT a.publisher_id, 
           a.publisher,
           b.max_count_books
    FROM a INNER JOIN b ON 1=1
    WHERE a.count_books = b.max_count_books
'''

task3 = pd.io.sql.read_sql(query, con = engine) 

print(f'Издательство, которое выпустило наибольшее число книг толще 50 страниц: {task3.publisher[0]}')

Издательство, которое выпустило наибольшее число книг толще 50 страниц: Penguin Books


- *Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.*

In [47]:
query = ''' 
    WITH a AS(
                SELECT authors.author_id,
                       author,
                       AVG(rating) AS avg_rating
                FROM books LEFT JOIN ratings ON books.book_id = ratings.book_id 
                           LEFT JOIN authors ON books.author_id = authors.author_id 
                GROUP BY authors.author_id, author
                HAVING COUNT(rating) > 50),
                
        b AS(
                SELECT max(avg_rating) as max_rating
                FROM a)

    SELECT a.author_id,
           a.author,
           b.max_rating
    FROM a INNER JOIN b on 1 = 1
    WHERE a.avg_rating = b.max_rating
'''

task4 = pd.io.sql.read_sql(query, con = engine)

print(f'Автор с самой высокой средней оценкой книг: {task4.author[0]}')

Автор с самой высокой средней оценкой книг: J.K. Rowling/Mary GrandPré


- *Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.*

In [48]:
query = ''' 
    WITH a AS(
                SELECT username, COUNT(rating) AS count_rating
                FROM ratings 
                GROUP BY username
                HAVING COUNT(rating) > 50),
                
         b AS(
                SELECT reviews.username,
                       COUNT(review_id) AS review_count
                       FROM reviews INNER JOIN a on a.username = reviews.username
                       GROUP BY reviews.username)

    SELECT avg(review_count)
    FROM b
'''

task5 = pd.io.sql.read_sql(query, con = engine)

print(f'Среднее количество обзоров от пользователей, которые поставили больше 50 оценок: {round(task5.avg[0],2)}')

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок: 24.33


---
### *Итоги*
- *Подключились к базе данных с помощью sqlalchemy*
- *Вывели первые строки таблиц*
- *Получили следующие выводы:*
    1. *819 книг вышло после 1 января 2000 года*
    2. *Для каждой книги посчитайте количество обзоров и среднюю оценку*
    3. *Издательство, которое выпустило наибольшее число книг толще 50 страниц: Penguin Books*
    4. *Автор с самой высокой средней оценкой книг: J.K. Rowling/Mary GrandPré*
    5. *Среднее количество обзоров от пользователей, которые поставили больше 50 оценок: 24.33*
  